# Step 1: Environment setup

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Importing dataset from kaggle to google colab


In [ ]:
# installing kaggle api
! pip install -q kaggle

In [ ]:
# create a directory as kaggle
! mkdir -p ~/.kaggle

In [ ]:
# importing kaggle api key to google colab
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# copy api key to kaggle directory
! cp kaggle.json ~/.kaggle

In [ ]:
# disable the api key
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog

100% 217M/218M [00:01<00:00, 137MB/s]
100% 218M/218M [00:01<00:00, 121MB/s]


In [ ]:
# unzip the dataset
! unzip -q /content/cat-and-dog.zip

In [ ]:
train_dir='/content/training_set/training_set'
test_dir='/content/test_set/test_set'

# Step 3: Build the model

In [ ]:
# reshape the images
img_shape=(128,128,3)

#Loading the pre-train model(MobileNetV2)

In [ ]:
base_model=tf.keras.applications.MobileNetV2(input_shape=img_shape,include_top=False,weights='imagenet')

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [ ]:
# frezzing the model
base_model.trainable=False

# Defining the custom head for network

In [ ]:
# checking output of the base model
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
# Adding global average pooling layer to set the output shape which related to our problem
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
# Defining the output/prediction layer
prediction_layer=tf.keras.layers.Dense(units=1,activation='sigmoid')(global_average_layer)

#Defining the transfer learning model


In [ ]:
model=tf.keras.models.Model(inputs=base_model.input,outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [ ]:
# compiling the model
opt=tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

# Create Data Generator(Data preprocessing)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Define an object
data_gen_train=ImageDataGenerator(rescale=1/255.0)
data_gen_test=ImageDataGenerator(rescale=1/255.0)

In [ ]:
train_generator=data_gen_train.flow_from_directory(directory=train_dir,target_size=(128,128),batch_size=128,class_mode='binary')
test_generator=data_gen_test.flow_from_directory(directory=test_dir,target_size=(128,128),batch_size=128,class_mode='binary')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


# Step 4: Train the model

In [ ]:
model.fit_generator(generator=train_generator,epochs=5,validation_data=test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/5
63/63 [==============================] - 109s 2s/step - loss: 0.8345 - accuracy: 0.4733 - val_loss: 0.6141 - val_accuracy: 0.6688
Epoch 2/5
63/63 [==============================] - 104s 2s/step - loss: 0.4910 - accuracy: 0.7806 - val_loss: 0.3747 - val_accuracy: 0.8655
Epoch 3/5
63/63 [==============================] - 104s 2s/step - loss: 0.3167 - accuracy: 0.8893 - val_loss: 0.2539 - val_accuracy: 0.9041
Epoch 4/5
63/63 [==============================] - 104s 2s/step - loss: 0.2284 - accuracy: 0.9237 - val_loss: 0.1927 - val_accuracy: 0.9273
Epoch 5/5
63/63 [==============================] - 103s 2s/step - loss: 0.1824 - accuracy: 0.9365 - val_loss: 0.1591 - val_accuracy: 0.9397


# Step 5: Fine tuning

In [ ]:
# unfrezzing the modl
base_model.trainable=True

In [ ]:
# check the total layer in base model
len(base_model.layers)

154

From 154 layer we will train the 100

In [ ]:
fine_tune_at=100

In [ ]:
# freezing the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  layer.traninable=False

In [ ]:
# compile the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics='accuracy')

In [ ]:
# training the model with fine tune
model.fit_generator(generator=train_generator,epochs=5,validation_data=test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/5
63/63 [==============================] - 528s 8s/step - loss: 0.1093 - accuracy: 0.9577 - val_loss: 0.1160 - val_accuracy: 0.9629
Epoch 2/5
63/63 [==============================] - 519s 8s/step - loss: 0.0156 - accuracy: 0.9958 - val_loss: 0.3401 - val_accuracy: 0.9308
Epoch 3/5
63/63 [==============================] - 520s 8s/step - loss: 0.0051 - accuracy: 0.9991 - val_loss: 0.4429 - val_accuracy: 0.9244
Epoch 4/5
63/63 [==============================] - 518s 8s/step - loss: 0.0027 - accuracy: 0.9995 - val_loss: 0.4774 - val_accuracy: 0.9318
Epoch 5/5
63/63 [==============================] - 509s 8s/step - loss: 0.0021 - accuracy: 0.9995 - val_loss: 1.1535 - val_accuracy: 0.8853
